# Training models
On the previously constructed dataset, we will train the following classificaiton models:
- VGG16
- MobileNet
- ResNet50

Further we will define our own model and train it.


In [4]:
import os
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, MobileNet, ResNet50  # Using ResNet50


# Parameters
data_dir = "../dat/images_augmented_cropped" 
target_size = (224, 224)
batch_size = 32
num_epochs = 10

validation_split = 0.2  # Percentage of data to use for validation

# Data Generator with Splitting
datagen = ImageDataGenerator(rescale=1./255, validation_split=validation_split)

# Load data with the split
train_data = datagen.flow_from_directory(
    data_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training' 
)

val_data = datagen.flow_from_directory(
    data_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation' 
)

# Model Creation with Pre-trained Weights
def create_model(model_type):
    if model_type == "VGG16":
        base_model = VGG16(include_top=False, weights='imagenet', input_shape=target_size + (3,))
    elif model_type == "MobileNet":
        base_model = MobileNet(include_top=False, weights='imagenet', input_shape=target_size + (3,))
    elif model_type == "ResNet50":
        base_model = ResNet50(include_top=False, weights='imagenet', input_shape=target_size + (3,))
    elif model_type == "OwnModel":
        base_model = keras.Sequential([
            keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=target_size),
            keras.layers.MaxPooling2D((2, 2)) ,
            keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
            keras.layers.MaxPooling2D((2, 2)),
            # ... Add more convolutional blocks if needed ...
            keras.layers.GlobalAveragePooling2D(),  # Bottleneck
            keras.layers.Dense(64, activation='relu'),  # Smaller dense layer
            keras.layers.Dropout(0.5),  # Regularization
            keras.layers.Dense(train_data.num_classes, activation='softmax'),
        ])
    else:
        raise ValueError("Invalid model_type")

    model = keras.Sequential([
        base_model,
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dense(train_data.num_classes, activation='softmax') 
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', 
                  metrics=['accuracy', 'top_k_categorical_accuracy@3', 'top_k_categorical_accuracy@5'])
    return model

# Training and Evaluation
models = {
    "VGG16": create_model("VGG16"),
    "MobileNet": create_model("MobileNet"),
    "ResNet50": create_model("ResNet50"),
    #"OwnModel": create_model("OwnModel"),
}

results = {}
for model_name, model in models.items():
    history = model.fit(train_data, epochs=num_epochs, validation_data=val_data)
    results[model_name] = history.history

    # Save model parameters
    model.save(f"../mod/{model_name}.h5")

# Save Metrics 
with open('model_metrics.txt', 'w') as f:
    for model_name, metrics in results.items():
        f.write(f"Metrics for {model_name}:\n")
        for metric, values in metrics.items():
            f.write(f"{metric}: {values}\n")

print("Training and Evaluation Complete!")


Found 50975 images belonging to 31 classes.
Found 12740 images belonging to 31 classes.
Epoch 1/10




TypeError: in user code:

    File "c:\Users\david\miniconda3\envs\gemstone\lib\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\david\miniconda3\envs\gemstone\lib\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\david\miniconda3\envs\gemstone\lib\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\david\miniconda3\envs\gemstone\lib\site-packages\keras\src\engine\training.py", line 1155, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "c:\Users\david\miniconda3\envs\gemstone\lib\site-packages\keras\src\engine\training.py", line 1249, in compute_metrics
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "c:\Users\david\miniconda3\envs\

In [ ]:
import itertools
from sklearn.metrics import confusion_matrix, accuracy_score

# Function for plotting confusion matrices
def plot_confusion_matrix(cm, classes, title='Confusion Matrix', normalize=False, cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Generate confusion matrix and calculate class-wise accuracy 
for model_name, model in models.items():
    y_pred = model.predict(val_data)
    y_pred = np.argmax(y_pred, axis=1)  # Convert probabilities to class labels
    y_true = val_data.classes

    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    plot_confusion_matrix(cm, classes=train_data.class_indices, title=f'Confusion Matrix - {model_name}')
    plt.show()

    # Class-wise accuracy
    class_accuracies = accuracy_score(y_true, y_pred, normalize=False) / val_data.n  # Per-class count
    for class_name, accuracy in zip(train_data.class_indices, class_accuracies):
        print(f"Class {class_name} Accuracy ({model_name}): {accuracy:.3f}")